In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [27]:
jobs_results = pd.read_csv("../../Data/jobs_results.csv")
jobs_results.head()

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,search_name,search_location,schedule_type,category
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,Language Fluency: English (Required)\nThe requ...,Deliver highly complex solutions with signific...,General Description of Available Benefits for ...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...,block-chain-1,USA,Full-time,Block Chain
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,NaN,NaN,NaN,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...,block-chain-1,USA,Contractor,Block Chain
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,Ability to parse and understand EVM (and/or So...,Tackling interesting problems - we are ingesti...,An accelerated growth trajectory - you will ha...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,block-chain-1,USA,Full-time,Block Chain
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",We’re looking for a capable generalist enginee...,You will be designing and building our core so...,"We’re wrapping up our presale for $JAM, so you...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,block-chain-1,USA,Full-time,Block Chain
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,NaN,NaN,NaN,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,block-chain-1,USA,Full-time,Block Chain


In [4]:
jobs_results['location'].unique()

array(['Anywhere', 'New York, NY', 'Los Angeles, CA', 'Livermore, CA',
       'Irving, TX', 'Mechanicsburg, PA', 'Texas', 'Milwaukee, WI',
       'United States', 'Dallas, TX', 'Chicago, IL',
       'Colorado Springs, CO', 'Chevy Chase, MD', 'Montreal, MO',
       'Cupertino, CA', 'Lexington, MA', 'Seattle, WA',
       'San Francisco, CA', 'Washington', 'Texas   (+5 others)',
       'Waltham, MA', 'Redmond, WA', 'North Reading, MA', 'Palo Alto, CA',
       'Milpitas, CA', 'Crownsville, MD', 'Boston, MA', 'San Mateo, CA',
       'Maryland', 'Annapolis Junction, MD', 'Annapolis, MD',
       'Catonsville, MD', 'Columbia, MD', 'Olathe, KS', 'Bentonville, AR',
       'Missouri', 'Rogers, AR', 'Beatrice, NE', 'Redwood City, CA',
       'Laurel, MD', 'Plano, TX', 'Oak Ridge, TN',
       'Irving, TX  (+1 other)', 'Bolinas, CA', 'Oklahoma City, OK',
       'United States   (+26 others)', 'Los Gatos, CA', 'Dearing, KS',
       'Hanover, MD', 'Fort Meade, MD', 'South San Francisco, CA',
       'A

In [5]:
density_of_data_science_jobs = jobs_results.copy()

# remove rows where location is anywhere
density_of_data_science_jobs = density_of_data_science_jobs.loc[density_of_data_science_jobs['location'] != 'Anywhere']

# remove if parenthesis are present (these don't have city info)
density_of_data_science_jobs = density_of_data_science_jobs.loc[~density_of_data_science_jobs['location'].str.contains('\(')]

# remove if they don't have commas (these don't have cities either)
density_of_data_science_jobs = density_of_data_science_jobs.loc[density_of_data_science_jobs['location'].str.contains(',')]


density_of_data_science_jobs['location'].unique()

array(['New York, NY', 'Los Angeles, CA', 'Livermore, CA', 'Irving, TX',
       'Mechanicsburg, PA', 'Milwaukee, WI', 'Dallas, TX', 'Chicago, IL',
       'Colorado Springs, CO', 'Chevy Chase, MD', 'Montreal, MO',
       'Cupertino, CA', 'Lexington, MA', 'Seattle, WA',
       'San Francisco, CA', 'Waltham, MA', 'Redmond, WA',
       'North Reading, MA', 'Palo Alto, CA', 'Milpitas, CA',
       'Crownsville, MD', 'Boston, MA', 'San Mateo, CA',
       'Annapolis Junction, MD', 'Annapolis, MD', 'Catonsville, MD',
       'Columbia, MD', 'Olathe, KS', 'Bentonville, AR', 'Rogers, AR',
       'Beatrice, NE', 'Redwood City, CA', 'Laurel, MD', 'Plano, TX',
       'Oak Ridge, TN', 'Bolinas, CA', 'Oklahoma City, OK',
       'Los Gatos, CA', 'Dearing, KS', 'Hanover, MD', 'Fort Meade, MD',
       'South San Francisco, CA', 'Arlington, VA', 'Ann Arbor, MI',
       'San Diego, CA', 'Miami, FL', 'Sunnyvale, CA', 'Berkeley, CA',
       'Littleton, CO', 'Atlanta, GA', 'Silver Spring, MD',
       'West Col

In [6]:
state_abb_and_names = {

', AL': ', Alabama',
', AK': ', Alaska',
', AZ': ', Arizona',
', AR': ', Arkansas',
', AS': ', American Samoa',
', CA': ', California',
', CO': ', Colorado',
', CT': ', Connecticut',
', DE': ', Delaware',
', DC': ', District of Columbia',
', FL': ', Florida',
', GA': ', Georgia',
', GU': ', Guam',
', HI': ', Hawaii',
', ID': ', Idaho',
', IL': ', Illinois',
', IN': ', Indiana',
', IA': ', Iowa',
', KS': ', Kansas',
', KY': ', Kentucky',
', LA': ', Louisiana',
', ME': ', Maine',
', MD': ', Maryland',
', MA': ', Massachusetts',
', MI': ', Michigan',
', MN': ', Minnesota',
', MS': ', Mississippi',
', MO': ', Missouri',
', MT': ', Montana',
', NE': ', Nebraska',
', NV': ', Nevada',
', NH': ', New Hampshire',
', NJ': ', New Jersey',
', NM': ', New Mexico',
', NY': ', New York',
', NC': ', North Carolina',
', ND': ', North Dakota',
', MP': ', Northern Mariana Islan',
', OH': ', Ohio',
', OK': ', Oklahoma',
', OR': ', Oregon',
', PA': ', Pennsylvania',
', PR': ', Puerto Rico',
', RI': ', Rhode Island',
', SC': ', South Carolina',
', SD': ', South Dakota',
', TN': ', Tennessee',
', TX': ', Texas',
', TT': ', Trust Territories',
', UT': ', Utah',
', VT': ', Vermont',
', VA': ', Virginia',
', VI': ', Virgin Islands',
', WA': ', Washington',
', WV': ', West Virginia',
', WI': ', Wisconsin',
', WY': ', Wyoming'

}      	

In [7]:
# Replace state abbreviations with state names
density_of_data_science_jobs['location'] = density_of_data_science_jobs['location'].replace(state_abb_and_names, regex=True)
density_of_data_science_jobs['location'].unique()

array(['New York, New York', 'Los Angeles, California',
       'Livermore, California', 'Irving, Texas',
       'Mechanicsburg, Pennsylvania', 'Milwaukee, Wisconsin',
       'Dallas, Texas', 'Chicago, Illinois', 'Colorado Springs, Colorado',
       'Chevy Chase, Maryland', 'Montreal, Missouri',
       'Cupertino, California', 'Lexington, Massachusetts',
       'Seattle, Washington', 'San Francisco, California',
       'Waltham, Massachusetts', 'Redmond, Washington',
       'North Reading, Massachusetts', 'Palo Alto, California',
       'Milpitas, California', 'Crownsville, Maryland',
       'Boston, Massachusetts', 'San Mateo, California',
       'Annapolis Junction, Maryland', 'Annapolis, Maryland',
       'Catonsville, Maryland', 'Columbia, Maryland', 'Olathe, Kansas',
       'Bentonville, Arkansas', 'Rogers, Arkansas', 'Beatrice, Nebraska',
       'Redwood City, California', 'Laurel, Maryland', 'Plano, Texas',
       'Oak Ridge, Tennessee', 'Bolinas, California',
       'Oklahoma Ci

In [8]:
# create a counts table
location_counts = density_of_data_science_jobs['location'].value_counts().reset_index()
location_counts.columns = ['location', 'count']

display(location_counts)


,location,count
0,"New York, New York",44
1,"Washington, District of Columbia",44
2,"San Francisco, California",28
3,"Annapolis Junction, Maryland",21
4,"Arlington, Virginia",19
...,...,...
144,"Irvine, California",1
145,"Berkeley, California",1
146,"Natick, Massachusetts",1
147,"Arlington, Texas",1


In [10]:
# use geopy nominatim to retrive coordinates using location names https://geopy.readthedocs.io/en/stable/
geolocator = Nominatim(user_agent="city_locator", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

location_counts['coordinate'] = location_counts['location'].apply(geocode)


display(location_counts.head())

,location,count,coordinate
0,"New York, New York",44,"(City of New York, New York, United States, (4..."
1,"Washington, District of Columbia",44,"(Washington, District of Columbia, United Stat..."
2,"San Francisco, California",28,"(San Francisco, CAL Fire Northern Region, Cali..."
3,"Annapolis Junction, Maryland",21,"(Annapolis Junction Town Center, Whitehurst, H..."
4,"Arlington, Virginia",19,"(Arlington County, Virginia, United States, (3..."


In [11]:
# arrange
for i, row in location_counts.iterrows():
    if row['coordinate']: # if row is not empty
        location_counts.loc[i, 'latitude'] = row['coordinate'].latitude
        location_counts.loc[i, 'longitude'] = row['coordinate'].longitude
    else:
        location_counts.loc[i, 'latitude'] = None
        location_counts.loc[i, 'longitude'] = None

# drop the coordinate column
location_counts = location_counts.drop('coordinate', axis=1)

# check
display(location_counts.head())

,location,count,latitude,longitude
0,"New York, New York",44,40.712728,-74.006015
1,"Washington, District of Columbia",44,38.895037,-77.036543
2,"San Francisco, California",28,37.779026,-122.419906
3,"Annapolis Junction, Maryland",21,39.125460,-76.795324
4,"Arlington, Virginia",19,38.876933,-77.089309


In [13]:
# drop the coordinate column
location_counts = location_counts.drop('count', axis=1)

# check
display(location_counts.head())

,location,latitude,longitude
0,"New York, New York",40.712728,-74.006015
1,"Washington, District of Columbia",38.895037,-77.036543
2,"San Francisco, California",37.779026,-122.419906
3,"Annapolis Junction, Maryland",39.125460,-76.795324
4,"Arlington, Virginia",38.876933,-77.089309


In [15]:
# Replace state abbreviations with state names
inverted_state_abb_and_names = {v: k for k, v in state_abb_and_names.items()}
location_counts['location_original'] = location_counts['location'].replace(inverted_state_abb_and_names, regex=True)
#location_counts['location_original'].unique()
display(location_counts.head())

,location,latitude,longitude,location_original
0,"New York, New York",40.712728,-74.006015,"New York, NY"
1,"Washington, District of Columbia",38.895037,-77.036543,"Washington, DC"
2,"San Francisco, California",37.779026,-122.419906,"San Francisco, CA"
3,"Annapolis Junction, Maryland",39.125460,-76.795324,"Annapolis Junction, MD"
4,"Arlington, Virginia",38.876933,-77.089309,"Arlington, VA"


In [16]:
# export
location_counts.to_csv('../../Data/location_counts_and_coords.csv', index=False)

In [28]:
# add coords to main dataframe
jobs_results_with_coords = pd.merge(jobs_results, location_counts, how='left', left_on='location', right_on='location_original')

# fix excess columns
jobs_results_with_coords = jobs_results_with_coords.drop(['location_original', 'location_y'], axis=1)
jobs_results_with_coords = jobs_results_with_coords.rename(columns={'location_x': 'location'})
jobs_results_with_coords['location'] = jobs_results_with_coords['location'].replace(state_abb_and_names, regex=True)


display(jobs_results_with_coords.head())

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,search_name,search_location,schedule_type,category,latitude,longitude
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,Language Fluency: English (Required)\nThe requ...,Deliver highly complex solutions with signific...,General Description of Available Benefits for ...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...,block-chain-1,USA,Full-time,Block Chain,NaN,NaN
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,NaN,NaN,NaN,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...,block-chain-1,USA,Contractor,Block Chain,NaN,NaN
2,Blockchain Analyst,Allium,"New York, New York",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,Ability to parse and understand EVM (and/or So...,Tackling interesting problems - we are ingesti...,An accelerated growth trajectory - you will ha...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,block-chain-1,USA,Full-time,Block Chain,40.712728,-74.006015
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, California",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",We’re looking for a capable generalist enginee...,You will be designing and building our core so...,"We’re wrapping up our presale for $JAM, so you...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,block-chain-1,USA,Full-time,Block Chain,34.053691,-118.242766
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, California","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,NaN,NaN,NaN,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,block-chain-1,USA,Full-time,Block Chain,37.682058,-121.768053


In [29]:
# export
jobs_results_with_coords.to_csv('../../Data/jobs_results_with_coords.csv', index=False)